In [1]:
import os
import altair as alt
import numpy as np
import seaborn as sns
import pandas as pd
import ipywidgets as widgets
import matplotlib.pyplot as plt 
import matplotlib.gridspec as gridspec
from vega_datasets import data
from ast import literal_eval
alt.renderers.enable('html')

C:\Users\kroep\anaconda3\envs\standard_env\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\kroep\anaconda3\envs\standard_env\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
C:\Users\kroep\anaconda3\envs\standard_env\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
C:\Users\kroep\anaconda3\envs\standard_env\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


RendererRegistry.enable('html')

In [2]:
def concatenate_files_in_folder(folder_path):
    files = os.listdir(folder_path)
    df_list = []

    for filename in files:
        df = pd.read_pickle(os.path.join(folder_path, filename))
        df_list.append(df)

    final_df = pd.concat(df_list, ignore_index=True)
    return final_df

In [3]:
folder_path = 'generated_df_data'
df = concatenate_files_in_folder(folder_path)

In [4]:
df.head()

,name,epoch,lr,classifier.weight,classifier.bias,confusion_matrix,train_errs,valid_errs,used_data_augmentation
0,alexnet_LearningRate0001_DataAugmentation,0,0.0001,"[[-0.024915101, -0.037159484, 0.016508136, 0.0...","[0.003112957, -0.025721112, -0.031407874, 0.01...","[[0, 0, 0, 0, 31, 0, 1], [0, 0, 0, 0, 48, 0, 1...",1.128776,0.947963,True
1,alexnet_LearningRate0001_DataAugmentation,1,0.0001,"[[-0.026296495, -0.043382544, 0.015737256, 0.0...","[0.0034747482, -0.026109323, -0.031466786, 0.0...","[[0, 0, 4, 0, 26, 0, 2], [0, 0, 5, 0, 40, 0, 4...",1.001434,0.891921,True
2,alexnet_LearningRate0001_DataAugmentation,2,0.0001,"[[-0.02675558, -0.048951242, 0.014891514, 0.02...","[0.0032282542, -0.026442608, -0.031296592, 0.0...","[[0, 0, 5, 0, 24, 0, 3], [0, 0, 8, 0, 37, 0, 4...",0.972001,0.866524,True
3,alexnet_LearningRate0001_DataAugmentation,3,0.0001,"[[-0.027105426, -0.053888228, 0.014519505, 0.0...","[0.002993757, -0.027054453, -0.030998982, 0.01...","[[0, 1, 7, 0, 21, 0, 3], [0, 3, 7, 0, 34, 0, 5...",0.943033,0.851340,True
4,alexnet_LearningRate0001_DataAugmentation,4,0.0001,"[[-0.027789587, -0.05788597, 0.0144679025, 0.0...","[0.0029351737, -0.027219074, -0.030957378, 0.0...","[[0, 4, 7, 0, 19, 0, 2], [0, 7, 4, 0, 33, 0, 5...",0.935439,0.844849,True


In [5]:
df['name'].unique()

array(['alexnet_LearningRate0001_DataAugmentation',
       'alexnet_LearningRate0001_NoDataAugmentation',
       'alexnet_LearningRate001_DataAugmentation',
       'alexnet_LearningRate001_NoDataAugmentation',
       'resnet50_LearningRate0001_DataAugmentation',
       'resnet50_LearningRate0001_NoDataAugmentation',
       'resnet50_LearningRate001_DataAugmentation',
       'resnet50_LearningRate001_NoDataAugmentation'], dtype=object)

In [6]:
conf_matr_dict = dict()

for model_name in df['name'].unique():
    conf_matr_dict[model_name] = df[df['name'].str.contains(model_name)].confusion_matrix

### Inspection of training porcess with altair

In [8]:
for model_name in df['name'].unique():
    conf_matr_series = df[df['name'].str.contains(model_name)].confusion_matrix
    conf_matr_series.reset_index(drop=True, inplace=True)
    conf_matr_dict[model_name] = conf_matr_series

In [9]:
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns
import numpy as np

@widgets.interact(model=['alexnet', 'resnet'], lr=[1e-4, 1e-3], data_transf=[True,False], 
                  epoch_matr=widgets.IntSlider(min=1, max=125, step=1, value=1))
def plot_epoch_acc(model, lr, data_transf, epoch_matr):
    
    model_name = ''
    model_name += 'resnet50' if model=='resnet' else 'alexnet'
    model_name += '_LearningRate001' if lr==1e-3 else '_LearningRate0001'
    model_name += '_DataAugmentation' if data_transf else '_NoDataAugmentation'
    
    print(model_name)

    epoch_length = len(conf_matr_dict[model_name])

    bal_acc_list = np.array([np.diag(conf_matr_dict[model_name][i])/conf_matr_dict[model_name][i].sum(axis=1) 
                             for i in range(epoch_length)]).T
    
    acc_list = np.array([np.diag(conf_matr_dict[model_name][i]).sum()/conf_matr_dict[model_name][i].sum() 
                         for i in range(epoch_length)])
    
    acc_list = acc_list.reshape(1, (len(acc_list)))

    # Create 3x3 sub plots
    gs = gridspec.GridSpec(3, 3)
    
    # Add subplots
    fig = plt.figure(figsize=(20, 12))
    ax1 = fig.add_subplot(gs[0, :])
    ax2 = fig.add_subplot(gs[1, :])
    ax3 = fig.add_subplot(gs[2, 0])
    ax4 = fig.add_subplot(gs[2, 1])
    ax5 = fig.add_subplot(gs[2, 2])
    
    sns.heatmap(bal_acc_list, cmap='Blues', ax=ax1)
    ax1.set_title('Accuracy per class and epoch')
    ax1.set_xlabel('Epochs of training')
    ax1.set_ylabel('Classes')

    sns.heatmap(acc_list, cmap='Blues', ax=ax2)
    ax2.set_title('Accuracy per epoch')
    ax2.set_xlabel('Epochs of training')
    ax2.set_ylabel('Accuracy')

    epoch_range = list(range(max(0, epoch_matr - 2), min(epoch_matr + 1, epoch_length)))
    for idx, ax in zip(epoch_range, [ax3, ax4, ax5]):
        sns.heatmap(conf_matr_dict[model_name][idx], cmap='Blues', annot=True, ax=ax)
        ax.set_title(f'Confusion matrix for epoch {idx+1}')
        ax.set_xlabel('Actual labels')
        ax.set_ylabel('Predicted labels')
    
    fig.tight_layout(pad=1.5)

    plt.show()

interactive(children=(Dropdown(description='model', options=('alexnet', 'resnet'), value='alexnet'), Dropdown(…